In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob

import matplotlib.pyplot as plt
%matplotlib notebook

The following query was used to select **obvious stars** from the Gaia-PS1 crossmatch table: 

    select ps1.*,
    sqrt(power(pmra,2) + power(pmdec,2)) as pm, 
    sqrt( power(pmra,2)*power(pmra_error, 2)/sqrt(power(pmra,2) + power(pmdec,2)) + power(pmdec,2)*power(pmdec_error, 2)/sqrt(power(pmra,2) + power(pmdec,2)) + 2*pmra*pmdec/sqrt(power(pmra,2) + power(pmdec,2))*pmra_pmdec_corr*pmra_error*pmdec_error) as pm_unc, gaia.parallax_over_error, gaia.phot_g_mean_mag, gaia.b
    from gaiadr2.panstarrs1_best_neighbour as ps1 
    inner join gaiadr2.gaia_source as gaia
    on ps1.source_id = gaia.source_id
    where gaia.astrometric_params_solved > 3
    and gaia.dec between -10 and -5
    order by ps1.original_ext_source_id

where the above query produced the file `dec_neg10-result.fits` and the declination range was adjusted to create all the other files.

## Create the loop to put all the objid into a single file

In [2]:
gaia_dir = "/Users/adamamiller/Desktop/PS1_fits/gaia_stars/"
gaia_files = glob.glob(gaia_dir + "tmp_dec*fits")
for file_num, gf in enumerate(gaia_files):
    gdat = fits.getdata(gf)
    def_stars = np.where(((gdat['pm']/gdat['pm_unc'] >= 8) | 
                         (gdat['parallax_over_error'] >= 11)) & 
                         (gdat['number_of_neighbours'] == 1) & 
                         (gdat['number_of_mates'] == 0))
    tmp_df = pd.DataFrame(gdat['original_ext_source_id'][def_stars], columns=['objid'])
    big_df = tmp_df.copy()
    big_df['pm_over_error'] = gdat['pm'][def_stars]/gdat['pm_unc'][def_stars]
    big_df['parallax_over_error'] = gdat['parallax_over_error'][def_stars]
    big_df['g_mag'] = gdat['phot_g_mean_mag'][def_stars]
    
    print(file_num, gf.split("/")[-1], len(gdat[def_stars]), 
          len(def_stars[0])/len(gdat))
    if file_num == 0:
        master_df = tmp_df.copy()
        master_big = big_df.copy()
    else:
        master_df = master_df.append(tmp_df, ignore_index=True)
        master_big = master_big.append(big_df, ignore_index=True)

0 tmp_dec_0-result.fits 5110330 0.2785525751203967
1 tmp_dec_10-result.fits 6201924 0.3102805221441797
2 tmp_dec_15-result.fits 6388247 0.33154770897609953
3 tmp_dec_20-result.fits 6295005 0.36693749488502947
4 tmp_dec_25-result.fits 6546555 0.38685338294280763
5 tmp_dec_30-result.fits 6623007 0.4031074726721968
6 tmp_dec_35-result.fits 6179851 0.3929004483242074
7 tmp_dec_40-result.fits 5282462 0.3654211750590662
8 tmp_dec_45-result.fits 5894353 0.3746364213648445
9 tmp_dec_5-result.fits 5605183 0.290260165596692
10 tmp_dec_50-result.fits 5475649 0.3535374150626149
11 tmp_dec_55-result.fits 4744850 0.34625376248167516
12 tmp_dec_60-result.fits 3734292 0.34639539491552845
13 tmp_dec_65-result.fits 2158858 0.3281036444089149
14 tmp_dec_70-result.fits 1530683 0.3572964494044215
15 tmp_dec_75-result.fits 929391 0.4103992666261003
16 tmp_dec_80-result.fits 511089 0.4380596394479867
17 tmp_dec_85-result.fits 147613 0.47411544786474125
18 tmp_dec_neg10-result.fits 5541367 0.2935190508049893


In [3]:
master_df.to_hdf(gaia_dir + "ps1_objid_4_gaia_stars.h5", "d1")
len(master_df)

125184032

In [4]:
master_big.to_hdf(gaia_dir + "ps1_gaia_mag_pm_plx.h5", "d1")
len(master_big)

125184032